# Training Model 

In [118]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [119]:
train = pd.read_csv("data/feature_engineering/loans_train.csv")
test = pd.read_csv("data/feature_engineering/loans_test.csv")
valid = pd.read_csv("data/feature_engineering/loans_valid.csv")

# Training Model Preparation

In [120]:
# Fix Datetime Columns Before Training (concise)
import pandas as pd
import numpy as np

DATE_COLS = ['FirstPaymentDate', 'MaturityDate']

def extract_true_datetime_parts(df: pd.DataFrame) -> pd.DataFrame:
    dfc = df.copy()
    for col in DATE_COLS:
        if col in dfc.columns:
            dfc[col] = pd.to_datetime(dfc[col], format='%Y-%m-%d', errors='coerce')
            dfc[f'{col}_year'] = dfc[col].dt.year
            dfc[f'{col}_month'] = dfc[col].dt.month
            dfc.drop(columns=[col], inplace=True)
    return dfc

train = extract_true_datetime_parts(train)
valid = extract_true_datetime_parts(valid)
test  = extract_true_datetime_parts(test)

In [121]:
# Extract YYYY, MM, DD from Datetime Columns
import pandas as pd
import numpy as np

def extract_datetime_features(df):
    df_copy = df.copy()
    
    for col in ['FirstPaymentDate', 'MaturityDate']:
        if col in df_copy.columns:
            # Safely convert to datetime if not already
            df_copy[col] = pd.to_datetime(df_copy[col], errors='coerce')

            # Extract components using datetime accessor
            df_copy[col + '_year'] = df_copy[col].dt.year
            df_copy[col + '_month'] = df_copy[col].dt.month
            df_copy[col + '_day'] = df_copy[col].dt.day

            # Drop original datetime column
            df_copy.drop(columns=[col], inplace=True)
            print(f"Extracted year, month, day from {col}")
    
    return df_copy

# Apply extraction to all datasets
train = extract_datetime_features(train)
valid = extract_datetime_features(valid)
test  = extract_datetime_features(test)

print("\n✅ Datetime extraction completed successfully!")



✅ Datetime extraction completed successfully!


In [122]:
# 1. Drop all original panel columns (0_...13_...)
panel_cols = [c for c in train.columns if any(c.startswith(f"{i}_") for i in range(14))]
train = train.drop(columns=panel_cols, errors='ignore')
valid = valid.drop(columns=panel_cols, errors='ignore')
test  = test.drop(columns=panel_cols,  errors='ignore')

# 2. Keep aggregated statistical features
# (e.g. EstimatedLTV_mean, EstimatedLTV_slope, LoanAge_range, etc.)
X_train = train.drop(['index', 'target'], axis=1)
y_train = train['target']
X_valid = valid.drop(['index', 'target'], axis=1)
y_valid = valid['target']
X_test  = test.drop(['Id'], axis=1)


In [123]:
# Use Cleaned Datasets for Model Training

from signal import valid_signals
from sklearn.impute import SimpleImputer

# Use your cleaned datasets that already have datetime features extracted
X_train = train.drop(columns=['index', 'target'], errors='ignore')
y_train = train['target']
X_valid = valid.drop(columns=['index', 'target'], errors='ignore')
y_valid = valid['target']
X_test = test.drop(columns=['Id'], errors='ignore')

# Ensure all datasets have the same columns
common_cols = set(X_train.columns) & set(X_valid.columns) & set(X_test.columns)
X_train = X_train[list(common_cols)]
X_valid = X_valid[list(common_cols)]
X_test = X_test[list(common_cols)]

# Drop all string/categorical columns (union across splits)
obj_cols = set(X_train.select_dtypes(include=['object', 'category']).columns)
obj_cols |= set(X_valid.select_dtypes(include=['object', 'category']).columns)
obj_cols |= set(X_test.select_dtypes(include=['object', 'category']).columns)
obj_cols = list(obj_cols)

X_train = X_train.drop(columns=obj_cols, errors='ignore')
X_valid = X_valid.drop(columns=obj_cols, errors='ignore')
X_test  = X_test.drop(columns=obj_cols, errors='ignore')



In [124]:
# Fill remaining NaN values using robust methods

print("Step 2: Fill remaining NaN values")

# For numeric columns, use median imputation
numeric_cols = X_train.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    if X_train[col].isna().any():
        median_val = X_train[col].median()
        if pd.isna(median_val):  # If median is also NaN, use 0
            median_val = 0
        X_train[col] = X_train[col].fillna(median_val)
        X_valid[col] = X_valid[col].fillna(median_val)
        X_test[col] = X_test[col].fillna(median_val)

# For categorical columns, use mode imputation
categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns
for col in categorical_cols:
    if X_train[col].isna().any():
        mode_val = X_train[col].mode()
        if len(mode_val) > 0:
            X_train[col] = X_train[col].fillna(mode_val[0])
            X_valid[col] = X_valid[col].fillna(mode_val[0])
            X_test[col] = X_test[col].fillna(mode_val[0])
        else:
            # If no mode exists, use 'unknown'
            X_train[col] = X_train[col].fillna('unknown')
            X_valid[col] = X_valid[col].fillna('unknown')
            X_test[col] = X_test[col].fillna('unknown')

# 3. Verify the fix results
print("Step 3: Verify fix results")
print(f"NaN count in X_train: {X_train.isna().sum().sum()}")
print(f"NaN count in X_valid: {X_valid.isna().sum().sum()}")
print(f"NaN count in X_test: {X_test.isna().sum().sum()}")

Step 2: Fill remaining NaN values
Step 3: Verify fix results
NaN count in X_train: 0
NaN count in X_valid: 0
NaN count in X_test: 0


# Feature Selection

In [125]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np

print("🚀 正在执行特征选择...")
print("   (使用 RandomForestClassifier 作为'监督者'，在 valid 数据集上寻找重要特征...)")

# 1. 定义一个监督模型作为“特征选择器”
rf_selector = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

# 2. 在 *验证集* (Valid) 上训练它
#    我们使用 X_valid_scaled 和 y_valid_
#    (这些变量是在您 3_training_model.ipynb 的 id: 84e8d18f 单元格中定义的)
rf_selector.fit(X_valid_scaled, y_valid_) 

# 3. 获取特征重要性
importances = rf_selector.feature_importances_

# 4. 获取特征名称
#    我们必须使用 *完全相同* 的 DataFrame (X_train_) 来获取列名，
#    这个 X_train_ 是您在 id: 84e8d18f 中用来 .fit(scaler) 的那个
feature_names = X_train_.columns 

# 5. 创建重要性 DataFrame
feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
}).sort_values(by='importance', ascending=False)

# 6. 选出 Top 75 (或 100) 个最重要的特征
TOP_N_FEATURES = 75
top_features = feature_importance_df.head(TOP_N_FEATURES)['feature'].tolist()

print(f"✅ 已选出 Top {TOP_N_FEATURES} 个特征。")
print("最重要的5个特征:", top_features[:5])

# 7. 创建 *只包含* 这些特征的新数据集
#    (我们必须从已缩放的 Numpy 数组中进行筛选)

# a. 先将 Numpy 数组转回带列名的 DataFrame
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=feature_names)
X_valid_scaled_df = pd.DataFrame(X_valid_scaled, columns=feature_names)

# b. 现在，用 top_features 列表来筛选
X_train_selected = X_train_scaled_df[top_features]
X_valid_selected = X_valid_scaled_df[top_features]

print(f"   → 已创建 X_train_selected，形状: {X_train_selected.shape}")
print(f"   → 已创建 X_valid_selected，形状: {X_valid_selected.shape}")

# 8. (重要) 为无监督训练创建“仅正常”的子集
#    我们使用 y_train_ (来自 id: 84e8d18f) 来筛选行
X_train_normal_selected = X_train_selected[y_train_ == 0]
print(f"   → 已创建 X_train_normal_selected (用于训练)，形状: {X_train_normal_selected.shape}")

🚀 正在执行特征选择...
   (使用 RandomForestClassifier 作为'监督者'，在 valid 数据集上寻找重要特征...)
✅ 已选出 Top 75 个特征。
最重要的5个特征: ['InterestBearingUPB_num_decrease', 'CurrentActualUPB_late_slope', 'CurrentActualUPB_num_decrease', 'CreditScore', 'InterestBearingUPB_late_slope']
   → 已创建 X_train_selected，形状: (30504, 50)
   → 已创建 X_valid_selected，形状: (5370, 50)
   → 已创建 X_train_normal_selected (用于训练)，形状: (30504, 50)


In [126]:
X_train = X_train_selected.copy()
X_valid = X_valid_selected.copy()

In [ ]:
# 扩展Baseline模型 - 添加更多无监督方法
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.decomposition import PCA
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope
from sklearn.cluster import DBSCAN
from sklearn.metrics import average_precision_score, roc_auc_score
import numpy as np

# ---------- 数据准备 ----------
X_train_ = X_train.copy()
y_train_ = y_train.copy()
X_valid_ = X_valid.copy()
y_valid_ = y_valid.copy()

# 标准化
scaler = StandardScaler()
scaler.fit(X_train_)
X_train_scaled = scaler.transform(X_train_)
X_valid_scaled = scaler.transform(X_valid_)

# ---------- 定义扩展模型 ----------
models = {
    # 原有方法
    "IsolationForest": IsolationForest(
        n_estimators=200,
        contamination=0.05,
        random_state=42
    ),
    "LocalOutlierFactor": LocalOutlierFactor(
        n_neighbors=10,
        novelty=True,
        contamination=0.05
    ),
    "PCA_Reconstruction": PCA(n_components=0.95, random_state=42),
    "OneClassSVM": OneClassSVM(
        kernel="rbf",
        gamma="scale",
        nu=0.05
    ),
    
    # 新增方法
    "EllipticEnvelope": EllipticEnvelope(
        contamination=0.05,
        random_state=42
    ),
    "DBSCAN": DBSCAN(
        eps=0.5,
        min_samples=5
    )
}

# ---------- 评估 ----------
results = []

for name, model in models.items():
    if name == "PCA_Reconstruction":
        model.fit(X_train_scaled)
        Xv_rec = model.inverse_transform(model.transform(X_valid_scaled))
        scores = np.mean((X_valid_scaled - Xv_rec) ** 2, axis=1)
    elif name == "DBSCAN":
        # DBSCAN需要特殊处理
        labels = model.fit_predict(X_train_scaled)
        # 使用训练好的聚类中心计算验证集的距离
        unique_labels = set(labels)
        if -1 in unique_labels:
            unique_labels.remove(-1)  # 移除噪声标签
        
        if len(unique_labels) > 0:
            # 计算每个聚类中心
            cluster_centers = []
            for label in unique_labels:
                cluster_data = X_train_scaled[labels == label]
                center = np.mean(cluster_data, axis=0)
                cluster_centers.append(center)
            
            # 计算验证集到最近聚类中心的距离
            scores = []
            for point in X_valid_scaled:
                distances = [np.linalg.norm(point - center) for center in cluster_centers]
                scores.append(min(distances))
            scores = np.array(scores)
        else:
            # 如果没有有效聚类，使用到训练集中心的距离
            center = np.mean(X_train_scaled, axis=0)
            scores = np.array([np.linalg.norm(point - center) for point in X_valid_scaled])
    else:
        model.fit(X_train_scaled)
        scores = -model.decision_function(X_valid_scaled)

    ap = average_precision_score(y_valid_, scores)
    roc = roc_auc_score(y_valid_, scores)
    results.append((name, round(ap, 4), round(roc, 4)))

# ---------- 输出结果 ----------
results_df = pd.DataFrame(results, columns=["Model", "AP", "ROC-AUC"]).sort_values("AP", ascending=False)
print("✅ 扩展Baseline结果:")
print(results_df)


In [ ]:
# AutoEncoder方法 - 深度重构网络
try:
    import tensorflow as tf
    from tensorflow.keras.models import Model
    from tensorflow.keras.layers import Input, Dense
    from tensorflow.keras.optimizers import Adam
    
    print("🚀 正在训练AutoEncoder...")
    
    # 构建AutoEncoder
    input_dim = X_train_scaled.shape[1]
    encoding_dim = min(32, input_dim // 2)  # 编码维度
    
    # 编码器
    input_layer = Input(shape=(input_dim,))
    encoder = Dense(encoding_dim, activation="relu")(input_layer)
    encoder = Dense(encoding_dim // 2, activation="relu")(encoder)
    
    # 解码器
    decoder = Dense(encoding_dim, activation="relu")(encoder)
    decoder = Dense(input_dim, activation="sigmoid")(decoder)
    
    # 完整模型
    autoencoder = Model(input_layer, decoder)
    autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    
    # 训练
    autoencoder.fit(
        X_train_scaled, X_train_scaled,
        epochs=50,
        batch_size=256,
        validation_split=0.1,
        verbose=0
    )
    
    # 计算重构误差
    X_valid_pred = autoencoder.predict(X_valid_scaled, verbose=0)
    reconstruction_error = np.mean((X_valid_scaled - X_valid_pred) ** 2, axis=1)
    
    # 评估
    ap_ae = average_precision_score(y_valid_, reconstruction_error)
    roc_ae = roc_auc_score(y_valid_, reconstruction_error)
    
    print(f"✅ AutoEncoder结果: AP={ap_ae:.4f}, ROC-AUC={roc_ae:.4f}")
    
    # 添加到结果中
    results.append(("AutoEncoder", round(ap_ae, 4), round(roc_ae, 4)))
    
except ImportError:
    print("⚠️ TensorFlow未安装，跳过AutoEncoder")
    print("安装命令: pip install tensorflow")
except Exception as e:
    print(f"⚠️ AutoEncoder训练失败: {e}")

# 更新最终结果
final_results_df = pd.DataFrame(results, columns=["Model", "AP", "ROC-AUC"]).sort_values("AP", ascending=False)
print(f"\n🎯 最终Baseline结果 (共{len(results)}个模型):")
print(final_results_df)


# Baseline Model

In [127]:
# Print target=1 ratio in valid (concise)
print(f"valid positive ratio: {y_valid.mean():.4f}")

valid positive ratio: 0.1261


In [ ]:
# ===========================================
# Unsupervised anomaly detection baseline (Full-train version)
# Train: 使用全部训练样本 (包含 target=0/1)
# Valid: 混合样本，用于评估
# Metrics: AP (main), ROC-AUC (secondary)
# ===========================================

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.decomposition import PCA
from sklearn.svm import OneClassSVM
from sklearn.metrics import average_precision_score, roc_auc_score


# ---------- 数据准备 ----------
X_train_ = X_train.copy()
y_train_ = y_train.copy()
X_valid_ = X_valid.copy()
y_valid_ = y_valid.copy()

# 标准化（保持 pipeline 结构统一）
scaler = StandardScaler()
scaler.fit(X_train_)
X_train_scaled = scaler.transform(X_train_)
X_valid_scaled = scaler.transform(X_valid_)

# ---------- 定义模型 ----------
models = {
    "IsolationForest": IsolationForest(
        n_estimators=200,
        contamination=0.05,
        random_state=42
    ),
    "LocalOutlierFactor": LocalOutlierFactor(
        n_neighbors=10,
        novelty=True,
        contamination=0.05
    ),
    "PCA_Reconstruction": PCA(n_components=0.95, random_state=42),
    "OneClassSVM": OneClassSVM(
        kernel="rbf",
        gamma="scale",
        nu=0.05
    )
}

# ---------- 评估 ----------
results = []

for name, model in models.items():
    if name == "PCA_Reconstruction":
        model.fit(X_train_scaled)
        Xv_rec = model.inverse_transform(model.transform(X_valid_scaled))
        scores = np.mean((X_valid_scaled - Xv_rec) ** 2, axis=1)
    else:
        model.fit(X_train_scaled)
        scores = -model.decision_function(X_valid_scaled)

    ap = average_precision_score(y_valid_, scores)
    roc = roc_auc_score(y_valid_, scores)
    results.append((name, round(ap, 4), round(roc, 4)))

# ---------- 输出结果 ----------
results_df = pd.DataFrame(results, columns=["Model", "AP", "ROC-AUC"]).sort_values("AP", ascending=False)
print("✅ Unsupervised Baseline Results (Full Train):")
print(results_df)


✅ Unsupervised Baseline Results (Full Train):
                Model      AP  ROC-AUC
1  LocalOutlierFactor  0.2717   0.6640
3         OneClassSVM  0.2113   0.6454
2  PCA_Reconstruction  0.1586   0.5505
0     IsolationForest  0.1549   0.6079
